In [3]:
import pandas as pd
import numpy as np
import itertools
import pickle
import torch
import matplotlib.pyplot as plt
from shutil import copyfile
%matplotlib inline
from statistics import mean
from scipy import stats, spatial
from ptflops import get_model_complexity_info
from shutil import copyfile

from main.layout import Layout
from main.models import Deeplab_ASPP_Layout
from main.algorithms import enum_layout, remove_redundancy, enum_layout_wo_rdt, metric_inference, init_S

# Enum coarse layout B=5

In [10]:
# Enum all layouts for given T (3 for NYUv2) and B (5 for coarse Deeplab_Resnet34)
T = 3 # segment_semantic, normal, depth_zbuffer
B = 6

layout_list = [] 
S0 = init_S(T, B) # initial state
L = Layout(T, B, S0) # initial layout
layout_list.append(L)

enum_layout_wo_rdt(L, layout_list)
len(layout_list)

70

In [4]:
# Simialr layouts
print(layout_list[43])
print(layout_list[44])

[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1}, {0, 2}], [{1}, {0, 2}]]
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1}, {0, 2}], [{1}, {2}, {0}]]


In [3]:
layout_list

[[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}]],
 [[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}]],
 [[{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]],
 [[{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]],
 [[{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]],
 [[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}]],
 [[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}]],
 [[{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}]],
 [[{1}, {0, 2}], [{1}, {2}, {0}], [{1}, {2}, {0}], [{1}, {2}, {0}], [{1}, {2}, {0}]],
 [[{1}, {0, 2}], [{1}, {0, 2}], [{1}, {2}, {0}], [{1}, {2}, {0}], [{1}, {2}, {0}]],
 [[{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {2}, {0}], [{1}, {2}, {0}]],
 [[{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {2}, {0}]],
 [[{2}, {0, 1}], [{2}, {

# Convert coarse layout to fine-grained layout for multi-task model

In [7]:
fined_B = 17
mapping = {0:[0], 1:[1,2,3], 2:[4,5,6,7], 3:[8,9,10,11,12,13], 4:[14,15,16], 5:[17]}

In [15]:
def coarse_to_fined(L, fined_B, mapping):
    S = []
    for idx in range(L.B):
        for times in mapping[idx]:
            S.append(L.state[idx])
    new_L = Layout(L.T, fined_B, S)
    return new_L

In [22]:
coarse_to_fined(layout_list[44], fined_B, mapping)

[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {2}, {0}], [{1}, {2}, {0}], [{1}, {2}, {0}]]

In [18]:
layout_list[44]

[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1}, {0, 2}], [{1}, {2}, {0}]]

# Convert fined 2task results to coarse

In [1]:
fined_2task_dir = './log/2task_coarse_NYUv2_mobilenet_v1/'
coarse_2task_dir = './log/2task_coarse_NYUv2_mobilenet/'

In [4]:
tasks = ['segment_semantic','normal','depth_zbuffer']
B = 6
mapping = {0:[0], 1:[1,2], 2:[3], 3:[4], 4:[5,6], 5:[7,8], 6:[9]}

for two_task in set(itertools.combinations(tasks, 2)):
    print(two_task)
    
    for idx in range(B+1):
        i = mapping[idx][0]
        log = '_'.join(two_task) + '_b' + str(i) + '.stdout'
        new_log = '_'.join(two_task) + '_b' + str(idx) + '.stdout'
        copyfile(fined_2task_dir + log, coarse_2task_dir + new_log)

('segment_semantic', 'normal')
('normal', 'depth_zbuffer')
('segment_semantic', 'depth_zbuffer')
